In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

# Set cache for cost saving
set_llm_cache(SQLiteCache("cache.db"))

# ChatOpenAI initialize
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# Few-shot examples
examples = [
    {
        "input": "Titanic",
        "output": """Movie: Titanic
Director: James Cameron
Cast: Leonardo DiCaprio, Kate Winslet, Billy Zane
Budget: $200 million
Box Office: $2.2 billion
Genre: Romance, Drama
Synopsis: A young aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic."""
    },
    {
        "input": "The Dark Knight",
        "output": """Movie: The Dark Knight
Director: Christopher Nolan
Cast: Christian Bale, Heath Ledger, Aaron Eckhart
Budget: $185 million
Box Office: $1.005 billion
Genre: Action, Crime, Drama
Synopsis: Batman faces the Joker, a criminal mastermind who wants to plunge Gotham City into anarchy."""
    },
    {
        "input": "Forrest Gump",
        "output": """Movie: Forrest Gump
Director: Robert Zemeckis
Cast: Tom Hanks, Robin Wright, Gary Sinise
Budget: $55 million
Box Office: $678.2 million
Genre: Drama, Comedy
Synopsis: The story follows several decades in the life of Forrest Gump, a slow-witted but kind-hearted man from Alabama who witnesses several defining historical events."""
    }
]

# Few-shot example prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# Few-shot chat message prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# final prompt template
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie database expert. When given a movie title, you provide detailed information about the movie in the exact format shown in the examples. Always follow the same structure and format."),
    few_shot_prompt,
    ("human", "{input}"),
])

# make chain
movie_chain = final_prompt | chat



AIMessageChunk(content='Movie: Forrest Gump\nDirector: Robert Zemeckis\nCast: Tom Hanks, Robin Wright, Gary Sinise\nBudget: $55 million\nBox Office: $678.2 million\nGenre: Drama, Comedy\nSynopsis: The story follows several decades in the life of Forrest Gump, a slow-witted but kind-hearted man from Alabama who witnesses several defining historical events.')

In [11]:
movie_chain.invoke({"input": "The Dark Knight"})

AIMessageChunk(content='Movie: The Dark Knight\nDirector: Christopher Nolan\nCast: Christian Bale, Heath Ledger, Aaron Eckhart\nBudget: $185 million\nBox Office: $1.005 billion\nGenre: Action, Crime, Drama\nSynopsis: Batman faces the Joker, a criminal mastermind who wants to plunge Gotham City into anarchy.')